In [4]:
#%%
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from PIL import Image


Pickle; Deserializing dictionary containing masks

In [30]:
Exp_2_pckl={}
path = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/Arttu/'
with open(path + 'Exp_design_2_ID.pkl', 'rb') as f:
    Exp_2_pckl = pickle.load(f)

Keys of nested dictionaries; Keys-indexes nested in the first internal dictionary


In [33]:
print('Length of the first counter-mask:', len(Exp_2_pckl[0]["masks"]))
print('Number of individual masks',len(Exp_2_pckl.keys()))
print('Variables for each mask:',Exp_2_pckl[0].keys())
print('Time:', (Exp_2_pckl[0]["time"]))


Length of the first counter-mask: 53
Number of individual masks 135
Variables for each mask: dict_keys(['masks', 'x', 'y', 'z', 'time', 'area'])
Time: [48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5, 55.0, 55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0, 60.5, 61.0, 61.5, 62.0, 62.5, 63.0, 63.5, 64.0, 64.5, 65.0, 65.5, 66.0, 66.5, 67.0, 67.5, 68.0, 68.5, 69.0, 69.5, 70.0, 70.5, 71.0, 71.5, 72.0, 72.5, 73.0, 73.5, 74.0]


CSV with measurement_id and well_id, to match with pickle

In [37]:
Exp_2_csv = pd.read_csv(path +'/Exp_design_2.csv', index_col=None) #loading the Arttu's csv file to match the measurement_id and well_id, to be matched with the pckl info

print('Variables for each row-mask:',Exp_2_csv.columns)
#print('Size of csv file:' )
print('Number of individual metadata sets (metadata line for each mask):',Exp_2_csv.shape[0])
print('Number of unique IDs in csv file:',len(Exp_2_csv["ID"].unique()))
print('Unique days:',(Exp_2_csv["day"].unique()))

print('Number of groups after measurement and well_id grouping:',(Exp_2_csv.groupby(['measurement_id', 'well_id'])).ngroups)


print(Exp_2_csv.loc[:,'day'].unique)



Variables for each row-mask: Index(['Unnamed: 0', 'seeding_density', 'cell_label', 'well_id',
       'measurement_id', 'matrix', 'ID', 'day', 'ID_running', 'location'],
      dtype='object')
Number of individual metadata sets (metadata line for each mask): 135
Number of unique IDs in csv file: 1
Unique days: [240715 240520 240522 240721 240724 240722 240723 240614 240307 240426
 240519 240521 240226 240301 240302 240303 240305 240311 240312 230417
 230418]
Number of groups after measurement and well_id grouping: 83
<bound method Series.unique of 0      240715
1      240715
2      240715
3      240715
4      240715
        ...  
130    230418
131    230418
132    230418
133    230418
134    230418
Name: day, Length: 135, dtype: int64>


Metadata from nd2 files

In [54]:
linda = Exp_2_csv
linda['grouped_id'] = linda.groupby(['measurement_id', 'well_id','cell_label','matrix']).ngroup()


for i in linda['grouped_id'].unique():
    x = linda[(linda['grouped_id']==i)]  
    print(i)
    print(x['cell_label'].values)



7
['MCF10A']
8
['DCIS.COM']
9
['MCF10AT']
10
['MCF10AT']
11
['MCF10AT']
12
['MCF10AT']
13
['MCF10AT']
14
['MCF10AT' 'MCF10AT']
15
['MCF10AT' 'MCF10AT']
16
['DCIS.COM' 'DCIS.COM']
17
['DCIS.COM' 'DCIS.COM' 'DCIS.COM']
18
['DCIS.COM' 'DCIS.COM' 'DCIS.COM']
19
['MCF10A' 'MCF10A' 'MCF10A']
0
['DCIS.COM' 'DCIS.COM' 'DCIS.COM']
1
['DCIS.COM' 'DCIS.COM' 'DCIS.COM' 'DCIS.COM']
2
['DCIS.COM' 'DCIS.COM' 'DCIS.COM' 'DCIS.COM']
3
['DCIS.COM' 'DCIS.COM' 'DCIS.COM' 'DCIS.COM']
4
['MCF10A' 'MCF10A' 'MCF10A' 'MCF10A']
5
['MCF10A' 'MCF10A' 'MCF10A' 'MCF10A']
6
['MCF10A' 'MCF10A' 'MCF10A' 'MCF10A']
20
['MCF10A']
21
['MCF10A']
22
['DCIS.COM']
23
['DCIS.COM']
24
['DCIS.COM']
25
['MCF10AT']
26
['MCF10A']
27
['DCIS.COM' 'DCIS.COM']
28
['DCIS.COM']
29
['MCF10A']
30
['MCF10A']
31
['MCF10A']
32
['MCF10A']
33
['DCIS.COM']
34
['DCIS.COM']
35
['DCIS.COM']
36
['DCIS.COM']
37
['MCF10AT']
38
['MCF10AT']
39
['MCF10AT']
40
['MCF10AT']
41
['DCIS.COM']
42
['DCIS.COM']
43
['DCIS.COM']
44
['MCF10A']
45
['DCIS.COM']
46
['M

In [ ]:
for material in unique_materials:
    for line in unique_lines:
        subset = Exp2[(Exp2['matrix'] == material) & (Exp2['cell_line'] == line)]
        duplicates = subset.duplicated(subset=['grouped_id'])
        if duplicates.any():
            print(f"Duplicate grouped_id found for {material} and {line}")


In [8]:
Exp_2_json = pd.read_json('C:/Users/srboval1/InstruProject/NikonPipes/dataStore/metalib.json') 
print(Exp_2_json.get(key="240521").get("m"))

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

print(search_json(240521))


0.660045336571487
0.660045336571487


In [10]:
print(Exp_2_json)

                name                                             240520  \
coords            []  [[[1090, 475], [1714, 1316]], [[939, 812], [17...   
matrix            []                                             IPN3mM   
cell              []  [T, T, DCIS.COM, DCIS.COM, DCIS.COM, MCF10A, M...   
n_cells           []                                                  1   
ignore            []                                                [6]   
stain             []                                                NaN   
dim                0                                                 3D   
incubation_time    0                                                 48   
seeding_density    0                [500, 500, 500, 500, 500, 500, 500]   
passage_number     0                                                NaN   
other              0                                              other   
dt               NaN                                             1800.0   
n_fields         NaN     

In [12]:

concentration_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
material_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
reverse_material_mapping = {
'2mgml':'collagen 2mg/ml',
'3mM':'IPN 2.5mM',
'40mM':'IPN 40mM'
}
cell_mapping = {
'MCF10A': '1',
'DCIS.COM': '2',
'MCF10A.DCIS.COM': '2',
'MCF10AT': '3'
}


Checking pixel size converstions to match

In [55]:
pixel_size = 0.32*1e-6

print('List of contour coordinates:',Exp_2_pckl[2]["masks"][10])
print('Linda calculated area [pixels]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10]))
print('re-calculated area in [m]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10])*(0.660535171355723**2))

print('Arttu calculated area:',Exp_2_pckl[2]["area"][10])

List of contour coordinates: [[[1106 1193]]

 [[1105 1194]]

 [[1104 1194]]

 ...

 [[1109 1193]]

 [[1108 1193]]

 [[1107 1193]]]
Linda calculated area [pixels]: 126034.0
re-calculated area in [m]: 54989.480215568066
Arttu calculated area: 54989.480215568066


In [56]:
parameter= 240521
Exp_2_json.get(key=str(parameter)).get("m")

json_days= Exp_2_json.keys()
csv_days = Exp_2_csv["day"].unique()

 
print('No of unique days in metafile:',len(Exp_2_json.keys()))
print('Unique days in metafile:',(Exp_2_json.keys()))


print('No of unique days in csv:',len(Exp_2_csv["day"].unique()))
print('Unique days in csv:',(Exp_2_csv["day"].unique()))


No of unique days in metafile: 25
Unique days in metafile: Index(['name', '240520', '240521', '240522', '240519', '230417', '240502',
       '240226', '240301', '240302', '240305', '240303', '240311', '240312',
       '230418', '240614', '240307', '240426', '240715', '240716', '240721',
       '240722', '240723', '240724', '240725'],
      dtype='object')
No of unique days in csv: 21
Unique days in csv: [240715 240520 240522 240721 240724 240722 240723 240614 240307 240426
 240519 240521 240226 240301 240302 240303 240305 240311 240312 230417
 230418]


Combining json and csv into metadata

In [58]:
#%%
#pixel_size = 0.32*1e-6
df_collecting = []
df_all = []

def search_csv(key_id):
    well = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'well_id'].iloc[0]
    measurement = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'measurement_id'].iloc[0]
    cell_line = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'cell_label'].iloc[0]
    material = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'matrix'].iloc[0]
    day = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'day'].iloc[0]
    return well, measurement, cell_line, material, day

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

for key in Exp_2_pckl.keys() :
    df_temp =[] 

    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time            
        mask = Exp_2_pckl[key]["masks"][i]
        time = Exp_2_pckl[key]["time"][i]
        
        well_id,measurement_id, cell, matrix, date = search_csv(key)

        area_pixels2 = cv2.contourArea(mask)        
        area_µm = area_pixels2*(search_json(date)**2)    

        perimeter = cv2.arcLength(mask, closed=True)*search_json(date) #perimeter for each contour
        compactness = 4 * math.pi * area_µm  #compactness (spheroid area/area of object with the identical perimeter)
        rect = cv2.minAreaRect(mask) #minimal rectangle
        width, height  = rect[1] #extracting the tuple containing rectangular dimensions
        aspect_ratio = width / height
        convexhull = cv2.convexHull(mask)
        convexhull_perimeter = cv2.arcLength(convexhull, closed=True)*search_json(date) #perimeter for each contour
        convexhull_area = cv2.contourArea(convexhull)*(search_json(date)**2)
        convexity_perimeter = convexhull_perimeter/perimeter
        convexity_area = convexhull_area/area_µm
        
        if i == 0:
            area_µm_zero = area_pixels2*(search_json(date)**2)    
        normalized_area_µm = area_µm / area_µm_zero

        
        info={}
        info = {
            "ID_running": key,
            "time_id": i+1,
            "incubation_time": time,
            "area_pixels**2":area_pixels2,
            "area_µm": area_µm,
            "normalized_area_µm":normalized_area_µm,
            "perimeter_µm": perimeter,
            "compactness": compactness,
            "aspect_ratio": aspect_ratio,
            "convex hull perimeter": convexhull_perimeter,
            "convex hull area": convexhull_area,
            "convexity_perimeter": convexity_perimeter,
            "convexity_area": convexity_area,
            "well_id":well_id,
            "measurement_id":measurement_id,
            "cell_line":cell,
            "matrix":matrix,
            "date":date
            }
        df_temp.append(pd.DataFrame([info]) )  #appending the info to the contours_info, which collects all timestemps
    
    df_collecting.append(pd.concat(df_temp,ignore_index=True)) #lists of dfs are contantenated and appended to df_collecting, thedf_temp is re-initialized

df_all=pd.concat(df_collecting, ignore_index=True)

C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\3618692324.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:, 'growth_rate'] = (specific_spheroid['area_µm'] - area0 )/ area0
C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\3618692324.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:, 'growth_rate'] = (specific_spheroid['area_µm'] - area0 )/ area0
C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\3618692324.py:83: SettingWithCopyWarning: 
A value

Additional parameters into metadata

In [62]:
df_all['spheroid_ID'] = df_all.groupby(['measurement_id', 'well_id','cell_line','matrix']).ngroup() #grouping time-stemps for each spheroid

df_collecting_normalized = []
df_all_normalized =[]

#Growth column ((size at t-size at zero)/size at zero)
for i in df_all['grouped_id'].unique(): #looping through time            
    specific_spheroid = df_all[(df_all['grouped_id'] == i)]
    min_time = np.argmin(specific_spheroid['incubation_time'])
    area0 = specific_spheroid.iloc[min_time]['area_µm']
    specific_spheroid.loc[:, 'growth_rate_%'] = ((specific_spheroid['area_µm'] - area0 )/ area0)*100
    df_collecting_normalized.append(pd.concat([specific_spheroid]))

df_all_normalized = pd.concat(df_collecting_normalized, ignore_index=True)

saving_path= 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2'
df_all_normalized.to_csv(saving_path+'/contours_exp2.csv')  

C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\2502271584.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:, 'growth_rate_%'] = ((specific_spheroid['area_µm'] - area0 )/ area0)*100
C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\2502271584.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:, 'growth_rate_%'] = ((specific_spheroid['area_µm'] - area0 )/ area0)*100
C:\Users\srboval1\AppData\Local\Temp\ipykernel_6440\2502271584.py:11: SettingWithCopyW

Visualizing the mask

In [38]:
#%%
save = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/TimeLapse/'
width, height = 2304, 2304

# Create a blank white image using Pillow and converting it into array


for key in Exp_2_pckl.keys() :
    time = Exp_2_pckl[key]["time"]
    
    blank_image = Image.new('RGB', (width, height), color = (255, 255, 255))
    blank_image_array = np.asarray(blank_image)

    new_image=np.zeros((height, width, 3), np.uint8)


    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time
        mask = Exp_2_pckl[key]["masks"][i]
        spheroid_id = key+1
        blank_image_array= cv2.drawContours(blank_image_array, Exp_2_pckl[key]["masks"][i], -1, (0, 0, 0), 1)
        #print( f'{unique_material}- {unique_line}; min: {min_value} and max {max_value}')

    plt.title(f'{matrix} - {cell_line}; spheroid id: {spheroid_id}') 
    plt.imshow(new_image)
    filename = f'{cell_line}_{matrix}_id_{id}.tiff'

    cv2.imwrite(save+filename, blank_image_array) #saving the contours

NameError: name 'cell_line' is not defined